# Modelling Choice Behaviour - Group E
## London Passenger Mode Choice
Students : DAMBREVILLE Nathan, DELPLANQUE Maxime, POULY Timothée

### **<u>I/ Imports</u>**
#### **1 - Libraries**

In [3]:
# %conda env create -f env/MCB_E.yml
# %pip install biogeme

import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from biogeme.expressions import Beta, Variable, log, exp
from biogeme.results_processing import get_pandas_estimated_parameters, html_output
from scipy.stats import norm

c:\Users\Administrateur\miniconda3\envs\dspap_project\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


#### **2 - Data**

In [4]:
DATA_FOLDER = 'data/'
data = pd.read_csv(DATA_FOLDER + 'lpmc09.dat', sep='\t')
data.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,13,1,1,1,4,3,1,5,0.0,1,...,0.241389,0.000000,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
1,43,10,0,0,3,3,6,1,1.0,1,...,0.072778,0.000000,0.344722,0.120556,1,0.167778,3.0,0.44,0.0,0.145695
2,46,12,0,0,4,3,1,5,0.0,1,...,0.136389,0.000000,0.070278,0.000000,0,0.072222,0.0,0.24,0.0,0.107692
3,53,12,1,3,4,3,2,1,1.0,1,...,0.082500,0.000000,0.061944,0.000000,0,0.062500,1.5,0.17,0.0,0.124444
4,65,13,1,3,3,5,1,5,0.0,1,...,0.050833,0.216667,0.590556,0.237778,2,0.863889,0.0,2.60,0.0,0.675884


### **<u>II/ Model 0</u>**

**Description:**
- This model includes alternative specific constant, cost and travel time for each alternative.
- Cost and travel time are associated with generic parameters.

#### **1 - Creation of the model**

In [ ]:
# Create the Biogeme database
database = db.Database("lpmc09", data)

# Define the base variables
travel_mode = Variable('travel_mode')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_access = Variable('dur_pt_access')
dur_pt_int = Variable('dur_pt_int')
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

# Create new variables
dur_pt_tot = dur_pt_bus + dur_pt_access + dur_pt_int
cost_drive = cost_driving_fuel + cost_driving_ccharge

# Define the ASC to be estimated
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

# Define the Betas to be estimated
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

# Define the utility functions per alternative
v_walking = dur_walking * beta_time
v_cycling = asc_cycling + dur_cycling * beta_time
v_pt = asc_pt + dur_pt_tot * beta_time + cost_transit * beta_cost
v_drive = asc_driving + dur_driving * beta_time + cost_drive * beta_cost

# Define the association between alternatives and utility functions
V = {1: v_walking,
     2: v_cycling,
     3: v_pt,
     4: v_drive}
logprob = models.loglogit(V, None, travel_mode)

# Initialisation of the Biogeme object
model_0 = bio.BIOGEME(database, logprob)
model_0.model_name = 'model_0'

#### **2 - Estimating the model's results**

In [6]:
results_model_0 = model_0.estimate()
general_statistics_model_0 = results_model_0.print_general_statistics()
print(general_statistics_model_0)

Number of estimated parameters             5
Sample size                                5000
Excluded observations                      0
Init log likelihood                        -4552.633
Final log likelihood                       -4552.633
Likelihood ratio test for the init. model  -0
Rho-square for the init. model             0
Rho-square-bar for the init. model         -0.0011
Akaike Information Criterion               9115.265
Bayesian Information Criterion             9147.851
Final gradient norm                        1.8474E-05
Bootstrapping time                         None


In [7]:
general_statistics_model_0

'Number of estimated parameters             5\nSample size                                5000\nExcluded observations                      0\nInit log likelihood                        -4552.633\nFinal log likelihood                       -4552.633\nLikelihood ratio test for the init. model  -0\nRho-square for the init. model             0\nRho-square-bar for the init. model         -0.0011\nAkaike Information Criterion               9115.265\nBayesian Information Criterion             9147.851\nFinal gradient norm                        1.8474E-05\nBootstrapping time                         None'

In [8]:
get_pandas_estimated_parameters(estimation_results=results_model_0)

,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,beta_time,-4.374888,0.164549,-26.587163,0.0
1,asc_cycling,-3.351221,0.099973,-33.521236,0.0
2,asc_pt,-0.641355,0.058471,-10.968706,0.0
3,beta_cost,-0.137783,0.013934,-9.888477,0.0
4,asc_driving,-0.735068,0.067722,-10.854174,0.0


#### **3 - Results analysis**
##### a - Overall Model Fit
- **Log-likelihood**: The initial log-likelihood is **-6931.472**, and the final log-likelihood is **-4552.633**. The improvement suggests the model fits the data better after estimation.
- **Likelihood Ratio Test**: The value is **4757.678**, which is high, indicating a significant improvement in model fit compared to the null model (no predictors).
- **Rho-square (Pseudo R²)**: Both the **Rho-square for the init. model (0.343)** and the **Rho-square-bar for the init. model (0.342)** suggest that about 34% of the variation in the dependent variable is explained by the model, which is a moderate level of explanatory power.
- **AIC and BIC**: The **Akaike Information Criterion (9115.265)** and **Bayesian Information Criterion (9147.851)** are provided for model comparison. Lower values indicate better fit, but without comparison to other models, their absolute values are less interpretable.

##### b - Interpretation of Parameter Signs
- **beta_time (-4.37)**: The negative sign indicates that as the time variable increases, the utility decreases. This is intuitive: people generally prefer options that take less time.
- **asc_cycling (-3.35)**: The negative sign for the alternative-specific constant (ASC) for cycling suggests that, all else being equal, individuals have a lower inherent preference for cycling compared to the base alternative.
- **asc_pt (-0.64)**: The negative ASC for public transport (PT) indicates a lower inherent preference for public transport compared to the base alternative.
- **beta_cost (-0.14)**: The negative sign for cost means that as the cost increases, the utility decreases. This is expected, as people prefer cheaper options.
- **asc_driving (-0.74)**: The negative ASC for driving suggests a lower inherent preference for driving compared to the base alternative.

##### c - Statistical Significance
- **Robust p-values**: All parameters have **p-values of 0.0**, which means they are **statistically significant** at any conventional level.
- **Robust t-statistics**: All t-statistics are far from zero (absolute values much greater than 2), further confirming the statistical significance of each parameter.

### **<u>III/ Model 1</u>**

**Description:**
- This model turns the parameter of time from a generic one into an alternative specific one.

#### **0 - Creation of the model**

In [9]:
# Define alternative-specific time Betas
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

# Utilities with alternative-specific time parameters
v_walking = dur_walking * beta_time_walk
v_cycling = asc_cycling + dur_cycling * beta_time_cycling
v_pt = asc_pt + dur_pt_tot * beta_time_pt + cost_transit * beta_cost
v_drive = asc_driving + dur_driving * beta_time_drive + cost_drive * beta_cost

# Association and estimation
V = {1: v_walking,
     2: v_cycling,
     3: v_pt,
     4: v_drive}
logprob = models.loglogit(V, None, travel_mode)

model_1 = bio.BIOGEME(database, logprob)
model_1.model_name = 'model_1'


#### **1 - Underlying assumption for alternative-specific time parameters**

Defining separate time parameters for each alternative implies that travellers value travel time differently depending on the mode.
For instance, a minute spent walking may be perceived as more onerous than a minute spent on public transport.
This specification lets the marginal disutility (or sensitivity) to travel time vary by mode, capturing mode-specific perceptions of time.

#### **2 - Estimating the model's results**

##### **a/ Coding the estimation**

In [36]:
# Estimate and display results
results_model_1 = model_1.estimate()
general_statistics_model_1 = results_model_1.print_general_statistics()

parameters_model_1 = get_pandas_estimated_parameters(estimation_results=results_model_1)
parameters_model_1

,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,beta_time_walk,-7.595577,0.409546,-18.546347,0.0
1,asc_cycling,-4.370240,0.196393,-22.252547,0.0
2,beta_time_cycling,-4.354759,0.457139,-9.526125,0.0
3,asc_pt,-2.486912,0.141213,-17.611026,0.0
4,beta_time_pt,-1.981675,0.171941,-11.525341,0.0
5,beta_cost,-0.154665,0.013395,-11.546452,0.0
6,asc_driving,-1.918849,0.140329,-13.673965,0.0
7,beta_time_drive,-3.456668,0.222864,-15.510187,0.0


##### **b/ Results interpretation**

- The `general statistics` of the model suggests that it has a small explanatory power It is only slightly better than the model 0, but still better.
- Compared to the original `beta_time` (~ -4.375), the `beta_time_walk` is almost two times bigger (~ -7.596). This suggests that walking becomes even less attractive as time increases than it normally does.
- `beta_time_cycling` is very close to the original. This calls for the same interpretation as the model 0's parameter.
- `beta_time_drive` is lower than the original. Meaning that driving will be less unattractive because of travel time increase than average.
- `beta_time_pt` is even lower which suggests an even lower unattraction to public transport because of travel time increase.
- `asc_pt` and `asc_driving` increase the basic unattractivity of these modes as they respectively tripled and quadrupled.
- `asc_cycling` however remains fairly close to the model 0 estimation (0: ~ -3.351 vs ~ -4.370 :1). The same goes for `beta_cost`  (0: ~ -0.138 vs ~ -0.155 :1).
- All parameters estimated in model 1 are `statistically significant` as they all have **Robust t-stats**<-10 (except `beta_time_cycling` which is at -9.5 -very close-) and their **Robust p-values** are 0.0.

#### **3 - Comparing `Model 0` and `Model 1`**

##### **a/ Choice of statistical test**

To compare **Model 0** and **Model 1**, we need to test whether the added parameters in Model 1 (alternative-specific ones) significantly improve the fit of the model compared to Model 0.
Both models are **nested** — Model 0 is a restricted version of Model 1 (obtained by constraining some parameters to be equal across alternatives).

The appropriate test is therefore a **Likelihood Ratio Test (LRT)**.

**Test statistic:**
[
LR = -2 \times [LL_0 - LL_1]
]
where

* (LL_0) = log-likelihood of Model 0,
* (LL_1) = log-likelihood of Model 1.

The statistic follows a **χ² (chi-square) distribution** with degrees of freedom equal to the difference in the number of parameters between the two models ((df = k_1 - k_0)).

**Null hypothesis (H₀):**
The more complex Model 1 does *not* provide a statistically significant improvement in fit over Model 0 — i.e. the parameters added in Model 1 are jointly equal to zero.
[
H_0: \beta_{\text{new}} = 0
]

**Alternative hypothesis (H₁):**
At least one of the new parameters improves the model fit.

**Expected result:**
Since Model 1 introduces alternative-specific parameters, we expect a better fit — that is, a higher log-likelihood and a significant LR statistic (p < 0.05).
Hence, Model 1 is expected to be preferred.

In [11]:
# --- Likelihood Ratio Test between Model 0 and Model 1 ---

import scipy.stats as stats
from functions.find_word_in_str import find_word_in_str

# Retrieve general statistics from both estimated models
LL0_start_index = find_word_in_str(total_str=general_statistics_model_0, key='Final log likelihood')[0][1] + 23
LL1_start_index = find_word_in_str(total_str=general_statistics_model_1, key='Final log likelihood')[0][1] + 23
LL0_end_index = LL0_start_index+8
LL1_end_index = LL1_start_index+8
LL0 = float(general_statistics_model_0[LL0_start_index:LL0_end_index])
LL1 = float(general_statistics_model_1[LL1_start_index:LL1_end_index])

# Retrieve number of estimated parameters for each model
k0_index = find_word_in_str(total_str=general_statistics_model_0, key='Number of estimated parameters')[0][1] + 13
k1_index = find_word_in_str(total_str=general_statistics_model_1, key='Number of estimated parameters')[0][1] + 13
k0 = int(general_statistics_model_0[k0_index])
k1 = int(general_statistics_model_1[k1_index])

# Compute the Likelihood Ratio (LR) statistic
LR = -2 * (LL0 - LL1)

# Degrees of freedom (difference in number of parameters)
dif = k1 - k0

# Compute the p-value using the chi-square distribution
p_value = 1 - stats.chi2.cdf(LR, dif)

# Print results
print(f"--- Likelihood Ratio Test ---")
print(f"Log-likelihood (Model 0): {LL0:.3f}")
print(f"Log-likelihood (Model 1): {LL1:.3f}")
print(f"LR statistic: {LR:.3f}")
print(f"Degrees of freedom: {dif}")
print(f"p-value: {p_value:.4f}")

# Decision rule at 5% significance level
if p_value < 0.05:
    print("Reject H0 → Model 1 significantly improves the fit. Model 1 is preferred.")
    model_pref = model_1
else:
    print("Fail to reject H0 → Model 1 does not significantly improve the fit. Keep Model 0.")
    model_pref = model_0


|Final log likelihood| was found 1 time(s) in the string.
|Final log likelihood| was found 1 time(s) in the string.
|Number of estimated parameters| was found 1 time(s) in the string.
|Number of estimated parameters| was found 1 time(s) in the string.
--- Likelihood Ratio Test ---
Log-likelihood (Model 0): -4552.630
Log-likelihood (Model 1): -4338.650
LR statistic: 427.960
Degrees of freedom: 3
p-value: 0.0000
Reject H0 → Model 1 significantly improves the fit. Model 1 is preferred.


##### **c/ Results interpretation**

There are two possible outcomes:

<u>Case 1 — Model 1 is preferred (Reject H₀):</u>

* The **LR statistic** is large enough (p < 0.05), meaning that the likelihood improvement due to the additional parameters is statistically significant.
* This indicates that allowing **alternative-specific parameters** captures real differences in travelers’ sensitivities across modes.
* **Model 1 becomes Model_pref**, i.e. the preferred model for the next steps.

<u>Case 2 — Model 1 is *not* preferred (Fail to reject H₀):</u>

* The **LR statistic** is small (p ≥ 0.05), meaning that the improvement in fit is not statistically significant.
* In that case, the more complex Model 1 does not justify its additional parameters.
* **Model 0 remains Model_pref**.

<u>Quantifying the degree of preference</u>

The **magnitude of the LR statistic** (and its corresponding **p-value**) tells you *how strongly* Model 1 is preferred:

* A **large LR value** (e.g. > 6 for df = 1, > 9 for df = 2) → strong evidence that Model 1 fits significantly better.
* A **small LR value** (close to 0) → almost no gain in explanatory power.

You can also look at **information criteria** (AIC, BIC) for a secondary check: Lower AIC/BIC values indicate a preferred model while penalizing for complexity.

### **<u>IV/ Model 2</u>**

**Description:**
- This model 

#### **1 - Creation of the model**

In [12]:
# add the additional variables
bus_scale = Variable('bus_scale')
female = Variable('female')

# Define the ASC to be estimated
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

# Define the Betas to be estimated
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_female = Beta('beta_female', 0, None, None, 0)
beta_bus_scale = Beta('beta_bus_scale', 0, None, None, 0)

# Define the utility functions
v_walking = dur_walking * beta_time_walk + beta_female * female
v_cycling = asc_cycling + dur_cycling * beta_time_cycling + beta_female * female
v_pt = asc_pt + dur_pt_tot * beta_time_pt + cost_transit * beta_cost + beta_bus_scale * bus_scale + beta_female * female
v_drive = asc_driving + dur_driving * beta_time_drive + cost_drive * beta_cost

# Define the association between alternatives and utility functions
V = {1: v_walking,
     2: v_cycling,
     3: v_pt,
     4: v_drive}
logprob2 = models.loglogit(V, None, travel_mode)

# Initialisation of the Biogeme object
biogeme = bio.BIOGEME(database, logprob2)
biogeme.model_name = 'model_2'

# Results
results_model_2 = biogeme.estimate()

parameters_model_2 = get_pandas_estimated_parameters(estimation_results=results_model_2)
parameters_model_2

,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,beta_time_walk,-7.604563,0.409873,-18.553451,0.000000
1,beta_female,0.077159,0.062418,1.236163,0.216398
2,asc_cycling,-4.372043,0.196647,-22.232929,0.000000
3,beta_time_cycling,-4.374024,0.458480,-9.540278,0.000000
4,asc_pt,-2.438799,0.147397,-16.545734,0.000000
5,beta_time_pt,-1.997623,0.173239,-11.531027,0.000000
6,beta_cost,-0.149366,0.013516,-11.051101,0.000000
7,beta_bus_scale,-0.090789,0.074401,-1.220269,0.222363
8,asc_driving,-1.874676,0.143715,-13.044408,0.000000
9,beta_time_drive,-3.520193,0.228099,-15.432753,0.000000


The model results highlight a strong sensitivity to travel time across all transport modes. The coefficient beta_time_walk = -7.6046 (p < 0.001) indicates that increases in walking time drastically reduce its attractiveness — it is the most negative parameter in the model, reflecting a pronounced aversion to long walking trips. Time variables for the other modes are also negative and highly significant: beta_time_cycling = -4.3704 (p < 0.001), beta_time_pt = -1.9976 (p < 0.001), and beta_time_drive = -3.5202 (p < 0.001). This suggests that while all modes are penalized by longer durations, walking and cycling times have the strongest deterrent effects, whereas time spent in public transport or driving is perceived as slightly less burdensome.

The cost coefficient is negative as expected (beta_cost = -0.1494, p < 0.001), confirming that individuals prefer cheaper alternatives, though its magnitude is smaller than that of time. The gender variable (beta_female = 0.0772, p = 0.216) is not statistically significant, indicating no meaningful difference between men and women in mode choice behavior.

Regarding the alternative-specific constants, asc_cycling = -4.3720 (p < 0.001), asc_pt = -2.4388 (p < 0.001), and asc_driving = -1.8747 (p < 0.001) are all negative and significant. This means that, all else being equal, individuals prefer walking (the reference mode) to other alternatives. Among the non-walking modes, cycling is the least attractive, followed by public transport and then driving.

Finally, the beta_bus_scale = -0.0908 (p = 0.222) variable — representing a discount or fare reduction scheme for some bus users — is not significant. This suggests that receiving a fare discount does not significantly increase the likelihood of choosing the bus, implying that travel time, comfort, and service reliability remain more influential factors than cost for bus use.

Overall, the model reveals a population highly sensitive to travel time, especially for active modes such as walking and cycling, moderately sensitive to cost, and structurally inclined toward walking, while other modes — including bus travel despite fare reductions — are perceived as less attractive alternatives.

#### **2 - Comparison between Model 1 and Model 2**

In [13]:
# --- Likelihood Ratio Test between Model 1 and Model 2 ---

# Retrieve general statistics from both estimated models
LL1_start_index = find_word_in_str(total_str=general_statistics_model_1, key='Final log likelihood')[0][1] + 23
LL2_start_index = find_word_in_str(total_str=results_model_2.print_general_statistics(), key='Final log likelihood')[0][1] + 23
LL1_end_index = LL1_start_index+8
LL2_end_index = LL2_start_index+8
LL1 = float(general_statistics_model_1[LL1_start_index:LL1_end_index])
LL2 = float(results_model_2.print_general_statistics()[LL2_start_index:LL2_end_index])

# Retrieve number of estimated parameters for each model
k1_index = find_word_in_str(total_str=general_statistics_model_1, key='Number of estimated parameters')[0][1] + 13
k2_index = find_word_in_str(total_str=results_model_2.print_general_statistics(), key='Number of estimated parameters')[0][1] + 13
k1 = int(general_statistics_model_1[k1_index])
k2 = int(results_model_2.print_general_statistics()[k2_index])

# Compute the Likelihood Ratio (LR) statistic
LR = -2 * (LL1 - LL2)

# Degrees of freedom (difference in number of parameters)
dif = k2 - k1

# Compute the p-value using the chi-square distribution
p_value = 1 - stats.chi2.cdf(LR, dif)

# Print results
print(f"--- Likelihood Ratio Test ---")
print(f"Log-Likelihood Model 1: {LL1}")
print(f"Log-Likelihood Model 2: {LL2}")
print(f"Likelihood Ratio (LR): {LR}")
print(f"Degrees of Freedom: {dif}")
print(f"P-value: {p_value}")

# Decision rule at 5% significance level
if p_value < 0.05:
    print("Reject H0 → Model 2 significantly improves the fit. Model 2 is preferred.")
    model_pref = biogeme
else:
    print("Fail to reject H0 → Model 2 does not significantly improve the fit. Keep Model 1.")
    model_pref = model_1

# Final preferred model results
print("Preferred Model Results:")
final_results = model_pref.estimate()
print(final_results.print_general_statistics())


|Final log likelihood| was found 1 time(s) in the string.
|Final log likelihood| was found 1 time(s) in the string.
|Number of estimated parameters| was found 1 time(s) in the string.
|Number of estimated parameters| was found 1 time(s) in the string.
--- Likelihood Ratio Test ---
Log-Likelihood Model 1: -4338.65
Log-Likelihood Model 2: -4337.21
Likelihood Ratio (LR): 2.8799999999991996
Degrees of Freedom: -7
P-value: nan
Fail to reject H0 → Model 2 does not significantly improve the fit. Keep Model 1.
Preferred Model Results:
Number of estimated parameters             8
Sample size                                5000
Excluded observations                      0
Init log likelihood                        -4338.658
Final log likelihood                       -4338.658
Likelihood ratio test for the init. model  -0
Rho-square for the init. model             0
Rho-square-bar for the init. model         -0.00184
Akaike Information Criterion               8693.316
Bayesian Information Criteri

### **<u>V/ Model 3</u>**

**Description:**
- This model includes an appropriate non-linear transformation of the `cost` variable.

#### **1 - Creation of the model**

##### Underlying assumption of the non-linear specification defined in this situation

When the price reaches a certain point, it becomes too expansive for the user whatever the price is after this level, using a logarithm will make the price less significative as the price will increase

In [14]:
data["cost_transit"].isna().sum()

np.int64(0)

In [15]:
# Define variables
travel_mode = Variable('travel_mode')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_access = Variable('dur_pt_access')
dur_pt_int = Variable('dur_pt_int')
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

# Create new variables
dur_pt_tot = dur_pt_bus + dur_pt_access + dur_pt_int
cost_drive = cost_driving_fuel + cost_driving_ccharge

# Define the ASC to be estimated
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

# Define the Betas to be estimated
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

# Box-Cox transformation of costs
lambda_boxcox = Beta('lambda_boxcox', 1.01, -10, 10, 0) # Can't put 1 as it creates a log and it is impossible due to values equal to 0 in the cost variable
boxcox_cost_transit = models.boxcox(cost_transit, lambda_boxcox)
boxcox_cost_drive = models.boxcox(cost_drive, lambda_boxcox)

# Define the utility functions
v_walking = dur_walking * beta_time_walk
v_cycling = asc_cycling + dur_cycling * beta_time_cycling
v_pt = asc_pt + dur_pt_tot * beta_time_pt + boxcox_cost_transit * beta_cost
v_drive = asc_driving + dur_driving * beta_time_drive + boxcox_cost_drive * beta_cost

# Define the association between alternatives and utility functions
V = {1: v_walking,
     2: v_cycling,
     3: v_pt,
     4: v_drive}

# Define the logit model
logprob = models.loglogit(V, None, travel_mode)

# Initialisation of the Biogeme object
model_3 = bio.BIOGEME(database, logprob)
model_3.model_name = 'model_3'

In [16]:
results_model_3 = model_3.estimate()
print(results_model_3.print_general_statistics())
parameters_model_3 = get_pandas_estimated_parameters(estimation_results=results_model_3)
print(parameters_model_3)

Number of estimated parameters             9
Sample size                                5000
Excluded observations                      0
Init log likelihood                        -4338.54
Final log likelihood                       -4338.54
Likelihood ratio test for the init. model  -0
Rho-square for the init. model             0
Rho-square-bar for the init. model         -0.00207
Akaike Information Criterion               8695.08
Bayesian Information Criterion             8753.734
Final gradient norm                        8.9679E-04
Bootstrapping time                         None
                Name     Value  Robust std err.  Robust t-stat.  \
0     beta_time_walk -7.695582         0.468092      -16.440306   
1        asc_cycling -4.399111         0.210551      -20.893374   
2  beta_time_cycling -4.486412         0.505014       -8.883744   
3             asc_pt -2.584011         0.144489      -17.883788   
4       beta_time_pt -2.046115         0.221129       -9.253044   
5      l

**Overall Model Fit**
- **Log-likelihood**: The initial and final log-likelihood are both **-4553.795**, which is unusual because the final log-likelihood should improve (become less negative) after estimation. This suggests a potential issue with the estimation process or the model specification.
- **Likelihood Ratio Test**: The value is **0**, which is unexpected and indicates no improvement over the null model. This is likely due to the identical init and final log-likelihoods.
- **Rho-square**: Both values are **0**, indicating that the model does not explain any additional variation compared to the null model. This is unusual and suggests a problem with the model or data.
- **AIC and BIC**: The values are **9119.59** and **9158.693**, respectively. These are not directly interpretable without comparison to other models, but given the other metrics, they suggest a poor fit.
- **Final Gradient Norm**: The value is **5.0376E-04**, which is close to zero, indicating convergence.


**Interpretation of Parameter Signs**
- **beta_time (-4.37)**: The negative sign indicates that as the time variable increases, the utility decreases. This is intuitive and expected: people prefer options that take less time.
- **asc_cycling (-3.36)**: The negative alternative-specific constant (ASC) for cycling suggests a lower inherent preference for cycling compared to the base alternative.
- **asc_pt (-0.72)**: The negative ASC for public transport (PT) indicates a lower inherent preference for public transport compared to the base alternative.
- **lambda_boxcox (0.78)**: This is the estimated lambda for the Box-Cox transformation of the cost variable. A value between 0 and 1 suggests a transformation between a log and a linear relationship.
- **beta_cost (-0.19)**: The negative sign for the cost parameter means that as the cost increases, the utility decreases. This is expected, as people prefer cheaper options.
- **asc_driving (-0.91)**: The negative ASC for driving suggests a lower inherent preference for driving compared to the base alternative.


**Statistical Significance**
- **Robust p-values**:
  - All parameters except `lambda_boxcox` and `beta_cost` have **p-values of 0.0**, indicating they are **statistically significant**
  - `lambda_boxcox` has a p-value of **0.001**, which is still statistically significant.
  - `beta_cost` has a p-value of **0.003**, which is also statistically significant.
- **Robust t-statistics**:
  - All t-statistics are far from zero (absolute values much greater than 2), confirming the statistical significance of each parameter.

In [17]:
results_pref = results_model_1
parameters_pref = parameters_model_1

In [18]:
# Extract estimated betas from both models
def get_betas_dict(results):
    params = results.getEstimatedParameters()
    return dict(zip(params.index, params['Value']))


def compute_chosen_probs(df, betas, boxcox=False, lambda_name='lambda_boxcox'):
    """
    df : DataFrame contenant les données brutes
    betas : dict contenant les valeurs des coefficients (Name -> Value)
    boxcox : booléen pour indiquer si on applique la transformation Box–Cox
    lambda_name : nom du paramètre lambda dans le dictionnaire betas
    """
    # Extraction des coefficients
    b_time_pt = betas['beta_time_pt']
    b_time_cycling = betas['beta_time_cycling']
    b_time_walk = betas['beta_time_walk']
    b_time_drive = betas['beta_time_drive']
    b_cost = betas['beta_cost']
    asc_cyc = betas['asc_cycling']
    asc_pt = betas['asc_pt']
    asc_drv = betas['asc_driving']
    lam = betas[lambda_name] if boxcox else None
    
    dur_pt_tot = df['dur_pt_bus'] + df['dur_pt_access'] + df['dur_pt_int']
    cost_drive = df['cost_driving_fuel'] + df['cost_driving_ccharge']

    # Cost transform
    if boxcox:
        eps = 1e-6
        x_pt = np.maximum(df['cost_transit'], eps)
        x_dr = np.maximum(cost_drive, eps)
        cost_pt = (x_pt**lam - 1)/lam if lam != 0 else np.log(x_pt)
        cost_dr = (x_dr**lam - 1)/lam if lam != 0 else np.log(x_dr)
    else:
        cost_pt = df['cost_transit']
        cost_dr = cost_drive

    # Utilities
    V = np.column_stack([
        b_time_walk * df['dur_walking'],
        asc_cyc + b_time_cycling * df['dur_cycling'],
        asc_pt + b_time_pt * dur_pt_tot + b_cost * cost_pt,
        asc_drv + b_time_drive * df['dur_driving'] + b_cost * cost_dr
    ])
    # Logit probabilities
    vmax = V.max(axis=1, keepdims=True)
    expV = np.exp(V - vmax)
    P = expV / expV.sum(axis=1, keepdims=True)

    # Probability of chosen alternative
    chosen = df['travel_mode'].astype(int).values - 1
    p_chosen = np.clip(P[np.arange(len(df)), chosen], 1e-300, 1.0)
    return p_chosen

In [19]:
# Compute chosen probabilities for both models
betas_pref = get_pandas_estimated_parameters(results_pref)
betas_3 = get_pandas_estimated_parameters(results_model_3)

betas_model_pref = dict(zip(betas_pref['Name'], betas_pref['Value']))
betas_model_3 = dict(zip(betas_3['Name'], betas_3['Value']))

p1 = compute_chosen_probs(data, betas_model_pref, boxcox=False)
p2 = compute_chosen_probs(data, betas_model_3, boxcox=True)

# Compute log probabilities and differences
logp1 = np.log(p1)
logp2 = np.log(p2)
r = logp1 - logp2

# Cox test statistic
N = len(r)
r_bar = np.mean(r)
s_r = np.std(r, ddof=1)
T = np.sqrt(N) * r_bar / s_r
p_value = 2 * (1 - norm.cdf(abs(T)))

print("=== Cox Test ===")
print(f"Mean diff in log-likelihoods per obs: {r_bar:.6f}")
print(f"Std dev: {s_r:.6f}")
print(f"T statistic: {T:.3f}")
print(f"p-value: {p_value:.5f}")

if p_value < 0.05:
    if r_bar > 0:
        print("→ Reject H0: Model Pref fits significantly better than Model 3.")
    else:
        print("→ Reject H0: Model 3 fits significantly better than Model Pref.")
else:
    print("→ Fail to reject H0: No significant difference in fit.")


=== Cox Test ===
Mean diff in log-likelihoods per obs: 0.005003
Std dev: 0.096713
T statistic: 3.658
p-value: 0.00025
→ Reject H0: Model Pref fits significantly better than Model 3.


### **<u>V/ Market Share</u>**

**Question 1:** Report the size and weight of each stratum in your sample
By dividing every value of the Table 1 by the total population (8673713) we can obtain the weight, as the size of each stratum correspond to each value of Table 1
After we have to calculate the proportion based on our database, and after we divide the london proportion by our database proportion to get the weight

Now we need to get the number of corresponding people in our initial database

In [20]:
data["age_over_41"] = (data["age"] > 40).astype(int) # Create age_over_41 variable
data_grouped = data.groupby(["age_over_41","female"]).size() # Group by age_over_41 and female

In [21]:
london_female_over_41 = 1765143
london_female_under_40 = 2599058
london_male_over_41 = 1633263
london_male_under_40 = 2676249
london_total = london_female_over_41 + london_female_under_40 + london_male_over_41 + london_male_under_40

data_female_over_41 = data_grouped[1,1]
data_female_under_40 = data_grouped[0,1]
data_male_over_41 = data_grouped[1,0]
data_male_under_40 = data_grouped[0,0]

In [22]:
prop_london_female_over_41 = london_female_over_41 / london_total
prop_london_female_under_40 = london_female_under_40 / london_total
prop_london_male_over_41 = london_male_over_41 / london_total
prop_london_male_under_40 = london_male_under_40 / london_total

In [23]:
prop_data_female_over_41 = data_female_over_41 / data_grouped.sum()
prop_data_female_under_40 = data_female_under_40 / data_grouped.sum()
prop_data_male_over_41 = data_male_over_41 / data_grouped.sum()
prop_data_male_under_40 = data_male_under_40 / data_grouped.sum()
print(prop_data_female_over_41, prop_data_female_under_40, prop_data_male_over_41, prop_data_male_under_40)

0.2404 0.2882 0.2186 0.2528


In [24]:
print("The size of each stratum in our database is:")
print("Female Over 41:", data_female_over_41)
print("Female Under 40:", data_female_under_40)
print("Male Over 41:", data_male_over_41)
print("Male Under 40:", data_male_under_40)
print("Total population in our database:", data_grouped.sum())

weight_female_over_41 = prop_london_female_over_41 / prop_data_female_over_41
weight_female_under_40 = prop_london_female_under_40 / prop_data_female_under_40
weight_male_over_41 = prop_london_male_over_41 / prop_data_male_over_41
weight_male_under_40 = prop_london_male_under_40 / prop_data_male_under_40
print("")
print("The weight of each stratum is:")
print("Weight Female Over 41:", weight_female_over_41)
print("Weight Female Under 40:", weight_female_under_40)
print("Weight Male Over 41:", weight_male_over_41)
print("Weight Male Under 40:", weight_male_under_40)

The size of each stratum in our database is:
Female Over 41: 1202
Female Under 40: 1441
Male Over 41: 1093
Male Under 40: 1264
Total population in our database: 5000

The weight of each stratum is:
Weight Female Over 41: 0.846526159950492
Weight Female Under 40: 1.0397213136760648
Weight Male Over 41: 0.8613921668262056
Weight Male Under 40: 1.2205185952462472


In [25]:
df_weight = pd.DataFrame({
    "stratum_name": ["Female Over 41", "Female Under 40", "Male Over 41", "Male Under 40"],
    'age_over_41': [1, 0, 1, 0],
    'female': [1, 1, 0, 0],
    "size": [data_female_over_41, data_female_under_40, data_male_over_41, data_male_under_40],
    'weight ': [weight_female_over_41, weight_female_under_40, weight_male_over_41, weight_male_under_40]
})
df_weight

,stratum_name,age_over_41,female,size,weight
0,Female Over 41,1,1,1202,0.846526
1,Female Under 40,0,1,1441,1.039721
2,Male Over 41,1,0,1093,0.861392
3,Male Under 40,0,0,1264,1.220519


As $Model_{pref}$ is Model 1, we need to calculate the probabilities for each alternative based on Model 1 results

In [26]:
# Adding the strata to the base data
data["strata"] = data.apply(lambda row:
    0 if (row["female"] == 1 and row["age_over_41"] == 1)
    else 1 if (row["female"] == 1 and row["age_over_41"] == 0)
    else 2 if (row["female"] == 0 and row["age_over_41"] == 1)
    else 3, axis=1)
data["strata_name"] = data.apply(lambda row:
    "Female Over 41" if (row["female"] == 1 and row["age_over_41"] == 1)
    else "Female Under 40" if (row["female"] == 1 and row["age_over_41"] == 0)
    else "Male Over 41" if (row["female"] == 0 and row["age_over_41"] == 1)
    else "Male Under 40", axis=1) 
data["weight"] = data.apply(lambda row:
    weight_female_over_41 if (row["female"] == 1 and row["age_over_41"] == 1)
    else weight_female_under_40 if (row["female"] == 1 and row["age_over_41"] == 0)
    else weight_male_over_41 if (row["female"] == 0 and row["age_over_41"] == 1)
    else weight_male_under_40 if (row["female"] == 0 and row["age_over_41"] == 0)
    else 1, axis=1)
data.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,age_over_41,strata,strata_name,weight
0,13,1,1,1,4,3,1,5,0.0,1,...,0,0.132222,0.0,0.50,0.0,0.065126,1,2,Male Over 41,0.861392
1,43,10,0,0,3,3,6,1,1.0,1,...,1,0.167778,3.0,0.44,0.0,0.145695,0,1,Female Under 40,1.039721
2,46,12,0,0,4,3,1,5,0.0,1,...,0,0.072222,0.0,0.24,0.0,0.107692,1,2,Male Over 41,0.861392
3,53,12,1,3,4,3,2,1,1.0,1,...,0,0.062500,1.5,0.17,0.0,0.124444,1,0,Female Over 41,0.846526
4,65,13,1,3,3,5,1,5,0.0,1,...,2,0.863889,0.0,2.60,0.0,0.675884,1,2,Male Over 41,0.861392


In [27]:
betas_model_1 = get_pandas_estimated_parameters(results_pref)
dict_betas_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Value']))
dict_std_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Robust std err.']))

n_obs = data.shape[0]  # Number of observations in your sample

data["dur_pt_tot"] = data['dur_pt_bus'] + data['dur_pt_access'] + data['dur_pt_int']
data["cost_drive"] = data['cost_driving_fuel'] + data['cost_driving_ccharge']

# Weight is included in the data DataFrame data for each individual in the column 'weight'
weight = data['weight']

# Estimated coefficients (betas) and their standard errors (replace with your actual values)
betas = {
    'b_time_walk': dict_betas_model_1['beta_time_walk'],       # Coefficient for walking time
    'asc_cyc': dict_betas_model_1['asc_cycling'],              # Alternative-specific constant for cycling
    'b_time_cycling': dict_betas_model_1['beta_time_cycling'], # Coefficient for cycling time
    'asc_pt': dict_betas_model_1['asc_pt'],                    # Alternative-specific constant for public transport
    'b_time_pt': dict_betas_model_1['beta_time_pt'],           # Coefficient for public transport time
    'b_cost': dict_betas_model_1['beta_cost'],                 # Coefficient for cost
    'asc_drv': dict_betas_model_1['asc_driving'],              # Alternative-specific constant for driving
    'b_time_drive': dict_betas_model_1['beta_time_drive'],     # Coefficient for driving time
}

std_errors = {
    'b_time_walk': dict_std_model_1['beta_time_walk'],       # Standard error for walking time coefficient
    'asc_cyc': dict_std_model_1['asc_cycling'],              # Standard error for cycling ASC
    'b_time_cycling': dict_std_model_1['beta_time_cycling'], # Standard error for cycling time coefficient
    'asc_pt': dict_std_model_1['asc_pt'],                    # Standard error for public transport ASC
    'b_time_pt': dict_std_model_1['beta_time_pt'],           # Standard error for public transport time coefficient
    'b_cost': dict_std_model_1['beta_cost'],                 # Standard error for cost coefficient
    'asc_drv': dict_std_model_1['asc_driving'],              # Standard error for driving ASC
    'b_time_drive': dict_std_model_1['beta_time_drive'],     # Standard error for driving time coefficient
}

# Function to calculate utilities for each alternative
def calculate_utilities(data, betas):
    """
    Calculate the deterministic part of the utility (V) for each alternative.

    Args:
        data (pd.DataFrame): DataFrame containing the variables.
        betas (dict): Dictionary of estimated coefficients.

    Returns:
        np.ndarray: Array of utilities for each alternative and observation.
    """
    V = np.column_stack([
        betas['b_time_walk'] * data['dur_walking'],  # Walking utility
        betas['asc_cyc'] + betas['b_time_cycling'] * data['dur_cycling'],  # Cycling utility
        betas['asc_pt'] + betas['b_time_pt'] * data['dur_pt_tot'] + betas['b_cost'] * data['cost_transit'],  # Public transport utility
        betas['asc_drv'] + betas['b_time_drive'] * data['dur_driving'] + betas['b_cost'] * data['cost_drive'],  # Driving utility
    ])
    return V

# Function to calculate choice probabilities using the multinomial logit model
def calculate_probabilities(V):
    """
    Calculate choice probabilities using the multinomial logit formula.

    Args:
        V (np.ndarray): Array of utilities for each alternative and observation.

    Returns:
        np.ndarray: Array of choice probabilities for each alternative and observation.
    """
    exp_V = np.exp(V)
    sum_exp_V = np.sum(exp_V, axis=1, keepdims=True)
    P = exp_V / sum_exp_V
    return P

# Function to calculate market shares
def calculate_market_shares(data, betas, weight):
    """
    Calculate the predicted market shares for each alternative.

    Args:
        data (pd.DataFrame): DataFrame containing the variables.
        betas (dict): Dictionary of estimated coefficients.
        weight (pd.Series): Series of weights for each observation.

    Returns:
        np.ndarray: Array of predicted market shares for each alternative.
    """
    V = calculate_utilities(data, betas)
    P = calculate_probabilities(V)
    market_shares = np.array([np.sum(weight * P[:, i], axis=0) for i in range(P.shape[1])])
    market_shares /= len(weight) # Divide by the total number of observations to get the average
    return market_shares

# Function to generate bootstrapped coefficients
def bootstrap_betas(betas, std_errors):
    """
    Generate bootstrapped coefficients by adding random noise based on standard errors.

    Args:
        betas (dict): Dictionary of estimated coefficients.
        std_errors (dict): Dictionary of standard errors for each coefficient.

    Returns:
        dict: Dictionary of bootstrapped coefficients.
    """
    bootstrapped_betas = {}
    for key in betas:
        bootstrapped_betas[key] = betas[key] + np.random.normal(0, std_errors[key])
    return bootstrapped_betas

# Bootstrapping process
R = 5000  # Number of bootstrap replications
market_shares_bootstrap = np.zeros((R, 4))  # 4 alternatives: walking, cycling, public transport, driving

for r in range(R):
    # Generate bootstrapped coefficients
    bootstrapped_betas = bootstrap_betas(betas, std_errors)

    # Calculate market shares for this bootstrap sample
    market_shares_bootstrap[r, :] = calculate_market_shares(data, bootstrapped_betas, weight)

# Calculate 90% confidence intervals
confidence_intervals = np.percentile(market_shares_bootstrap, [5, 95], axis=0)

# Display the results in a table
df_market_shares = pd.DataFrame({
    "Mode": ["Walking", "Cycling", "Public Transport", "Driving"],
    "Market Share": market_shares_bootstrap.mean(axis=0),
    "90% CI Lower": confidence_intervals[0, :],
    "90% CI Upper": confidence_intervals[1, :]
})
df_market_shares

,Mode,Market Share,90% CI Lower,90% CI Upper
0,Walking,0.168881,0.144304,0.195391
1,Cycling,0.030751,0.019937,0.044795
2,Public Transport,0.349160,0.292200,0.407919
3,Driving,0.451209,0.385941,0.515573


Results Question 2: Inside df_market_shares

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mode</th>
      <th>Market Share</th>
      <th>90% CI Lower</th>
      <th>90% CI Upper</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Walking</td>
      <td>0.168711</td>
      <td>0.144431</td>
      <td>0.194904</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Cycling</td>
      <td>0.030703</td>
      <td>0.020153</td>
      <td>0.044021</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Public Transport</td>
      <td>0.349032</td>
      <td>0.294101</td>
      <td>0.408647</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Driving</td>
      <td>0.451554</td>
      <td>0.386645</td>
      <td>0.515989</td>
    </tr>
  </tbody>
</table>
</div>

#### **Question 3:** 
Compare the market shares predicted by $Model_{pref}$ with the weighted market shares computed using the actual choices.

In [28]:
actual_walking_share = 0
actual_bike_share = 0
actual_pt_share = 0
actual_drive_share = 0

for i in range(len(data)):
    if data.loc[i, "travel_mode"] == 1:
        actual_walking_share += data.loc[i, "weight"]
    elif data.loc[i, "travel_mode"] == 2:
        actual_bike_share += data.loc[i, "weight"]
    elif data.loc[i, "travel_mode"] == 3:
        actual_pt_share += data.loc[i, "weight"]
    elif data.loc[i, "travel_mode"] == 4:
        actual_drive_share += data.loc[i, "weight"]
        
actual_walking_share /= len(data)
actual_bike_share /= len(data)
actual_pt_share /= len(data)
actual_drive_share /= len(data)

print("Actual Market Shares (weighted):")
print("Walking:", actual_walking_share)
print("Cycling:", actual_bike_share)
print("Public Transport:", actual_pt_share)
print("Driving:", actual_drive_share)

# Display the results in a table with actual market shares
df_market_shares = pd.DataFrame({
    "Mode": ["Walking", "Cycling", "Public Transport", "Driving"],
    "Predicted Market Share": market_shares_bootstrap.mean(axis=0),
    "90% CI Lower": confidence_intervals[0, :],
    "90% CI Upper": confidence_intervals[1, :],
    "Actual Market Share": [actual_walking_share, actual_bike_share, actual_pt_share, actual_drive_share],
})
df_market_shares["Delta (Predicted - Actual)"] = df_market_shares["Predicted Market Share"] - df_market_shares["Actual Market Share"]
df_market_shares["% Change"] = (df_market_shares["Delta (Predicted - Actual)"] / df_market_shares["Actual Market Share"]) * 100
df_market_shares

Actual Market Shares (weighted):
Walking: 0.1710824582994761
Cycling: 0.030858157556110724
Public Transport: 0.35254210519189516
Driving: 0.44551727895250726


,Mode,Predicted Market Share,90% CI Lower,90% CI Upper,Actual Market Share,Delta (Predicted - Actual),% Change
0,Walking,0.168881,0.144304,0.195391,0.171082,-0.002202,-1.287046
1,Cycling,0.030751,0.019937,0.044795,0.030858,-0.000108,-0.348670
2,Public Transport,0.349160,0.292200,0.407919,0.352542,-0.003382,-0.959376
3,Driving,0.451209,0.385941,0.515573,0.445517,0.005692,1.277550


### **<u>VI/ Forecasting </u>**

Consider the following scenarios: (i) an increase of car cost by 15%;
and (ii) a decrease of public transport cost by 15%.

**Question 1:** Report the market shares predicted by Modelpref for each scenario. Do they match your
expectations? Compare the predicted market shares with the original market shares.

In [29]:
# Scenario i : Increase of car cost by 15%, based on model 1 (the preferred model)

data_scenario_i = data.copy()

# Adding the strata to the base data
data_scenario_i["strata"] = data_scenario_i.apply(lambda row:
    0 if (row["female"] == 1 and row["age_over_41"] == 1)
    else 1 if (row["female"] == 1 and row["age_over_41"] == 0)
    else 2 if (row["female"] == 0 and row["age_over_41"] == 1)
    else 3, axis=1)
data_scenario_i["strata_name"] = data_scenario_i.apply(lambda row:
    "Female Over 41" if (row["female"] == 1 and row["age_over_41"] == 1)
    else "Female Under 40" if (row["female"] == 1 and row["age_over_41"] == 0)
    else "Male Over 41" if (row["female"] == 0 and row["age_over_41"] == 1)
    else "Male Under 40", axis=1) 
data_scenario_i["weight"] = data_scenario_i.apply(lambda row:
    weight_female_over_41 if (row["female"] == 1 and row["age_over_41"] == 1)
    else weight_female_under_40 if (row["female"] == 1 and row["age_over_41"] == 0)
    else weight_male_over_41 if (row["female"] == 0 and row["age_over_41"] == 1)
    else weight_male_under_40 if (row["female"] == 0 and row["age_over_41"] == 0)
    else 1, axis=1)

betas_model_1 = get_pandas_estimated_parameters(results_pref)
dict_betas_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Value']))
dict_std_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Robust std err.']))

n_obs = data_scenario_i.shape[0]  # Number of observations in your sample

### HERE WE MODIFY THE COST DRIVE VARIABLE
data_scenario_i["dur_pt_tot"] = data_scenario_i['dur_pt_bus'] + data_scenario_i['dur_pt_access'] + data_scenario_i['dur_pt_int']
data_scenario_i["cost_drive"] = (data_scenario_i['cost_driving_fuel'] + data_scenario_i['cost_driving_ccharge']) * 1.15  # Increase driving cost by 15%

# Weight is included in the data DataFrame data for each individual in the column 'weight'
weight = data_scenario_i['weight']

# Estimated coefficients (betas) and their standard errors (replace with your actual values)
betas = {
    'b_time_walk': dict_betas_model_1['beta_time_walk'],       # Coefficient for walking time
    'asc_cyc': dict_betas_model_1['asc_cycling'],              # Alternative-specific constant for cycling
    'b_time_cycling': dict_betas_model_1['beta_time_cycling'], # Coefficient for cycling time
    'asc_pt': dict_betas_model_1['asc_pt'],                    # Alternative-specific constant for public transport
    'b_time_pt': dict_betas_model_1['beta_time_pt'],           # Coefficient for public transport time
    'b_cost': dict_betas_model_1['beta_cost'],                 # Coefficient for cost
    'asc_drv': dict_betas_model_1['asc_driving'],              # Alternative-specific constant for driving
    'b_time_drive': dict_betas_model_1['beta_time_drive'],     # Coefficient for driving time
}

std_errors = {
    'b_time_walk': dict_std_model_1['beta_time_walk'],       # Standard error for walking time coefficient
    'asc_cyc': dict_std_model_1['asc_cycling'],              # Standard error for cycling ASC
    'b_time_cycling': dict_std_model_1['beta_time_cycling'], # Standard error for cycling time coefficient
    'asc_pt': dict_std_model_1['asc_pt'],                    # Standard error for public transport ASC
    'b_time_pt': dict_std_model_1['beta_time_pt'],           # Standard error for public transport time coefficient
    'b_cost': dict_std_model_1['beta_cost'],                 # Standard error for cost coefficient
    'asc_drv': dict_std_model_1['asc_driving'],              # Standard error for driving ASC
    'b_time_drive': dict_std_model_1['beta_time_drive'],     # Standard error for driving time coefficient
}



# Bootstrapping process
R = 5000  # Number of bootstrap replications
market_shares_bootstrap = np.zeros((R, 4))  # 4 alternatives: walking, cycling, public transport, driving

for r in range(R):
    # Generate bootstrapped coefficients
    bootstrapped_betas = bootstrap_betas(betas, std_errors)

    # Calculate market shares for this bootstrap sample
    market_shares_bootstrap[r, :] = calculate_market_shares(data_scenario_i, bootstrapped_betas, weight)

# Calculate 90% confidence intervals
confidence_intervals = np.percentile(market_shares_bootstrap, [5, 95], axis=0)

# Display the results in a table
df_market_shares = pd.DataFrame({
    "Mode": ["Walking", "Cycling", "Public Transport", "Driving"],
    "Market Share": market_shares_bootstrap.mean(axis=0),
    "90% CI Lower": confidence_intervals[0, :],
    "90% CI Upper": confidence_intervals[1, :]
})
df_market_shares



,Mode,Market Share,90% CI Lower,90% CI Upper
0,Walking,0.169610,0.145337,0.196046
1,Cycling,0.031080,0.020358,0.044480
2,Public Transport,0.354184,0.299241,0.412974
3,Driving,0.445126,0.381253,0.507974


In [30]:
# Scenario ii : Decrease of PT cost by 15%, based on model 1 (the preferred model)

data_scenario_ii = data.copy()

# Adding the strata to the base data
data_scenario_ii["strata"] = data_scenario_ii.apply(lambda row:
    0 if (row["female"] == 1 and row["age_over_41"] == 1)
    else 1 if (row["female"] == 1 and row["age_over_41"] == 0)
    else 2 if (row["female"] == 0 and row["age_over_41"] == 1)
    else 3, axis=1)
data_scenario_ii["strata_name"] = data_scenario_ii.apply(lambda row:
    "Female Over 41" if (row["female"] == 1 and row["age_over_41"] == 1)
    else "Female Under 40" if (row["female"] == 1 and row["age_over_41"] == 0)
    else "Male Over 41" if (row["female"] == 0 and row["age_over_41"] == 1)
    else "Male Under 40", axis=1) 
data_scenario_ii["weight"] = data_scenario_ii.apply(lambda row:
    weight_female_over_41 if (row["female"] == 1 and row["age_over_41"] == 1)
    else weight_female_under_40 if (row["female"] == 1 and row["age_over_41"] == 0)
    else weight_male_over_41 if (row["female"] == 0 and row["age_over_41"] == 1)
    else weight_male_under_40 if (row["female"] == 0 and row["age_over_41"] == 0)
    else 1, axis=1)

betas_model_1 = get_pandas_estimated_parameters(results_pref)
dict_betas_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Value']))
dict_std_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Robust std err.']))

n_obs = data_scenario_ii.shape[0]  # Number of observations in your sample

### HERE WE MODIFY THE PT COST VARIABLE
data_scenario_ii["cost_transit"] = data_scenario_ii["cost_transit"] * 0.85  # Decrease PT cost by 15%
data_scenario_ii["dur_pt_tot"] = data_scenario_ii['dur_pt_bus'] + data_scenario_ii['dur_pt_access'] + data_scenario_ii['dur_pt_int'] 
data_scenario_ii["cost_drive"] = data_scenario_ii['cost_driving_fuel'] + data_scenario_ii['cost_driving_ccharge'] 

# Weight is included in the data DataFrame data for each individual in the column 'weight'
weight = data_scenario_ii['weight']

# Estimated coefficients (betas) and their standard errors (replace with your actual values)
betas = {
    'b_time_walk': dict_betas_model_1['beta_time_walk'],       # Coefficient for walking time
    'asc_cyc': dict_betas_model_1['asc_cycling'],              # Alternative-specific constant for cycling
    'b_time_cycling': dict_betas_model_1['beta_time_cycling'], # Coefficient for cycling time
    'asc_pt': dict_betas_model_1['asc_pt'],                    # Alternative-specific constant for public transport
    'b_time_pt': dict_betas_model_1['beta_time_pt'],           # Coefficient for public transport time
    'b_cost': dict_betas_model_1['beta_cost'],                 # Coefficient for cost
    'asc_drv': dict_betas_model_1['asc_driving'],              # Alternative-specific constant for driving
    'b_time_drive': dict_betas_model_1['beta_time_drive'],     # Coefficient for driving time
}

std_errors = {
    'b_time_walk': dict_std_model_1['beta_time_walk'],       # Standard error for walking time coefficient
    'asc_cyc': dict_std_model_1['asc_cycling'],              # Standard error for cycling ASC
    'b_time_cycling': dict_std_model_1['beta_time_cycling'], # Standard error for cycling time coefficient
    'asc_pt': dict_std_model_1['asc_pt'],                    # Standard error for public transport ASC
    'b_time_pt': dict_std_model_1['beta_time_pt'],           # Standard error for public transport time coefficient
    'b_cost': dict_std_model_1['beta_cost'],                 # Standard error for cost coefficient
    'asc_drv': dict_std_model_1['asc_driving'],              # Standard error for driving ASC
    'b_time_drive': dict_std_model_1['beta_time_drive'],     # Standard error for driving time coefficient
}



# Bootstrapping process
R = 5000  # Number of bootstrap replications
market_shares_bootstrap = np.zeros((R, 4))  # 4 alternatives: walking, cycling, public transport, driving

for r in range(R):
    # Generate bootstrapped coefficients
    bootstrapped_betas = bootstrap_betas(betas, std_errors)

    # Calculate market shares for this bootstrap sample
    market_shares_bootstrap[r, :] = calculate_market_shares(data_scenario_ii, bootstrapped_betas, weight)

# Calculate 90% confidence intervals
confidence_intervals = np.percentile(market_shares_bootstrap, [5, 95], axis=0)

# Display the results in a table
df_market_shares = pd.DataFrame({
    "Mode": ["Walking", "Cycling", "Public Transport", "Driving"],
    "Market Share": market_shares_bootstrap.mean(axis=0),
    "90% CI Lower": confidence_intervals[0, :],
    "90% CI Upper": confidence_intervals[1, :]
})
df_market_shares



,Mode,Market Share,90% CI Lower,90% CI Upper
0,Walking,0.168077,0.143760,0.194331
1,Cycling,0.030199,0.019733,0.043496
2,Public Transport,0.355221,0.297244,0.414978
3,Driving,0.446502,0.382087,0.512309


Les résultats se ressemblent vraiment beaucoup. Globalement, les gens prennent un tout petit peu moins la voiture dans le scénario i (attendu), et un tout petit peu plus les transports en commun dans le scénario ii (attendu aussi). On aurait pu s'attendre à une plus grande hausse que les 1 à 2% obtenus.

**Question 2:** Which scenario is the most effective policy if the goal is to decrease the share of car?
Explain why.

ça a l'air d'être le scénario ii, car en réduisant le coût des transports en commun, des usagers de la voiture décident de changer de mode. Cela attire plus dans ce cas que dans le cas du scénario i. 

**Question 3:** Calculate the total public transportation revenue in each scenario. Which scenario reports
the highest total revenue? Explain why. Is it higher than the total revenue obtained
without any of the policies? Can you explain why?

In [54]:
# Calculation of the total public transport revenue oustide of a scenario
total_pt_revenue = 0
for i in range(len(data)):
    if data.loc[i, "travel_mode"] == 3:
        total_pt_revenue += data.loc[i, "cost_transit"] * data.loc[i, "weight"]
print("Total Public Transport Revenue Outside of scenarios:", total_pt_revenue * (london_total/5000))


Total Public Transport Revenue Outside of scenarios: 6287959.503890217


In [53]:
# Calculation of the total public transport revenue in scenario i depending on the market shares
total_pt_revenue = 0
for i in range(len(data)):
    if data_scenario_i.loc[i, "travel_mode"] == 3:
        total_pt_revenue += data_scenario_i.loc[i, "cost_transit"] * data_scenario_i.loc[i, "weight"]
print("Total Public Transport Revenue in scenario i:", total_pt_revenue * (london_total/5000))



Total Public Transport Revenue in scenario i: 6287959.503890217


In [55]:
# Calculation of the total public transport revenue in scenario ii. 
total_pt_revenue = 0
for i in range(len(data_scenario_ii)):
    if data_scenario_ii.loc[i, "travel_mode"] == 3:
        total_pt_revenue += data_scenario_ii.loc[i, "cost_transit"] * data_scenario_ii.loc[i, "weight"]
print("Total Public Transport Revenue in Scenario ii (Decrease of PT cost by 15%):", total_pt_revenue * (london_total/5000))


Total Public Transport Revenue in Scenario ii (Decrease of PT cost by 15%): 5344765.578306692


The highest revenue in public transportation is in scenario i. Indeed, the cost is reduced in scenario ii. (attendu)

**Question 4:** Calculate the average value of time for car and public transportation. Comment on the
obtained results.

In [37]:
# Calculate the average value of time for cars and pt based on model 1 (the preferred model)
vot_pt =  dict_betas_model_1['beta_time_pt'] / dict_betas_model_1['beta_cost']
vot_driving =  dict_betas_model_1['beta_time_drive'] / dict_betas_model_1['beta_cost']
print("Average Value of Time (in currency unit per time unit):")

print("Public Transport VOT:", vot_pt)
print("Driving VOT:", vot_driving)

Average Value of Time (in currency unit per time unit):
Public Transport VOT: 12.812723639926624
Driving VOT: 22.349448118744196


For car ~22 -> high value of time 

For PT ~12 -> low value of time

**Question 5:** Compute the direct and cross aggregate elasticities of car cost and public transport cost.
Comment on the obtained results. Report the normalization factors.

In [43]:
# Compute the direct disaggregate elasticities of car cost and PT cost
dict_betas_model_1 = dict(zip(betas_model_1['Name'], betas_model_1['Value']))

beta_ = {
    'b_time_walk': dict_betas_model_1['beta_time_walk'],       # Coefficient for walking time
    'asc_cyc': dict_betas_model_1['asc_cycling'],              # Alternative-specific constant for cycling
    'b_time_cycling': dict_betas_model_1['beta_time_cycling'], # Coefficient for cycling time
    'asc_pt': dict_betas_model_1['asc_pt'],                    # Alternative-specific constant for public transport
    'b_time_pt': dict_betas_model_1['beta_time_pt'],           # Coefficient for public transport time
    'b_cost': dict_betas_model_1['beta_cost'],                 # Coefficient for cost
    'asc_drv': dict_betas_model_1['asc_driving'],              # Alternative-specific constant for driving
    'b_time_drive': dict_betas_model_1['beta_time_drive'],     # Coefficient for driving time
    
}

def compute_direct_disaggregate_elasticities(data, beta_):
    dur_pt_tot = data['dur_pt_bus'] + data['dur_pt_access'] + data['dur_pt_int']
    cost_drive = data['cost_driving_fuel'] + data['cost_driving_ccharge']

    V = np.column_stack([
        beta_['b_time_walk'] * data['dur_walking'],
        beta_['asc_cyc'] + beta_['b_time_cycling'] * data['dur_cycling'],
        beta_['asc_pt'] + beta_['b_time_pt'] * dur_pt_tot + beta_['b_cost'] * data['cost_transit'],
        beta_['asc_drv'] + beta_['b_time_drive'] * data['dur_driving'] + beta_['b_cost'] * cost_drive
    ])
    # Logit probabilities
    vmax = V.max(axis=1, keepdims=True)
    expV = np.exp(V - vmax)
    P = expV / expV.sum(axis=1, keepdims=True)
    # Direct disaggregate elasticities
    elasticities = np.zeros_like(P)
    for j in range(P.shape[1]):
        if j == 2:  # Public Transport
            elasticities[:, j] = beta_['b_cost'] * data['cost_transit'] * (1 - P[:, j])
        elif j == 3:  # Driving
            elasticities[:, j] = beta_['b_cost'] * cost_drive * (1 - P[:, j])
        else:
            elasticities[:, j] = 0  # No cost variable for walking and cycling
    return elasticities

elasticities = compute_direct_disaggregate_elasticities(data, beta_)

# Direct aggregate elasticities for PT and Driving with weighted average
weight = data['weight']
direct_agg_elasticity_pt = np.sum(weight * elasticities[:, 2]) / np.sum(weight)
direct_agg_elasticity_driving = np.sum(weight * elasticities[:, 3]) / np.sum(weight)
print("Direct Aggregate Elasticity of Public Transport Cost:", direct_agg_elasticity_pt)
print("Direct Aggregate Elasticity of Driving Cost:", direct_agg_elasticity_driving)
print("Normalization factor (average weight):", weight.mean())


Direct Aggregate Elasticity of Public Transport Cost: -0.13808913702535236
Direct Aggregate Elasticity of Driving Cost: -0.2326889377196007
Normalization factor (average weight): 1.0


In [42]:
# Compute the cross disaggregate elasticities of car cost and PT cost

def compute_cross_disaggregate_elasticities(data, beta_):
    dur_pt_tot = data['dur_pt_bus'] + data['dur_pt_access'] + data['dur_pt_int']
    cost_drive = data['cost_driving_fuel'] + data['cost_driving_ccharge']

    V = np.column_stack([
        beta_['b_time_walk'] * data['dur_walking'],
        beta_['asc_cyc'] + beta_['b_time_cycling'] * data['dur_cycling'],
        beta_['asc_pt'] + beta_['b_time_pt'] * dur_pt_tot + beta_['b_cost'] * data['cost_transit'],
        beta_['asc_drv'] + beta_['b_time_drive'] * data['dur_driving'] + beta_['b_cost'] * cost_drive
    ])

    # Logit probabilities
    vmax = V.max(axis=1, keepdims=True)
    expV = np.exp(V - vmax)
    P = expV / expV.sum(axis=1, keepdims=True)
    # Cross disaggregate elasticities
    elasticities = np.zeros_like(P)
    for j in range(P.shape[1]):
        for k in range(P.shape[1]):
            if j != k:
                if j == 2 and k == 3:  # PT choice with respect to Driving cost
                    elasticities[:, j] = -beta_['b_cost'] * cost_drive * P[:, j]
                elif j == 3 and k == 2:  # Driving choice with respect to PT cost
                    elasticities[:, j] = -beta_['b_cost'] * data['cost_transit'] * P[:, j]
                else:
                    elasticities[:, j] = 0  # No cost variable for walking and cycling
    return elasticities

elasticities_cross = compute_cross_disaggregate_elasticities(data, beta_)

# Cross aggregate elasticities for PT and Driving with weighted average
weight = data['weight']
cross_agg_elasticity_pt = np.sum(weight * elasticities_cross[:, 2]) / np.sum(weight)
cross_agg_elasticity_driving = np.sum(weight * elasticities_cross[:, 3]) / np.sum(weight)
print("Cross Aggregate Elasticity of Public Transport Cost:", cross_agg_elasticity_pt)
print("Cross Aggregate Elasticity of Driving Cost:", cross_agg_elasticity_driving)
print("Normalization factor (average weight):", weight.mean())

Cross Aggregate Elasticity of Public Transport Cost: 0.19725740282288173
Cross Aggregate Elasticity of Driving Cost: 0.10576818289835842
Normalization factor (average weight): 1.0


In [38]:
# report the normalization factor used in the elasticity calculations
print("Normalization factor (average weight):", weight.mean())


Normalization factor (average weight): 1.0
